In [1]:
from gensim import corpora, models, similarities
from gensim.models import hdpmodel, ldamodel
from itertools import izip
from nltk.corpus import stopwords
from pandas.core.series import Series
import math
import pdb
from spacy.en import English
from nltk import sent_tokenize
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
parser = English()
lmtzr = WordNetLemmatizer()
porter = PorterStemmer() 

/Users/himavarsha/Envs/sklearn/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
documents = []
with open('template_topic_modelling.txt','r') as f:
    documents = f.read().splitlines()
    

In [3]:
stoplist = stopwords.words("english") + ['blackberry','it\'s','i','a','an','is','my','to','it','the','oven']


In [4]:
import nltk
texts = []
dp_pairs = ["dobj","advcl","ccomp","nmod","compound", "acl", "acomp", "csubj", "amod"]
freq_dict = dict()
for example in documents:
    sentences = nltk.sent_tokenize(example)
    for sentence in sentences:
        try:
            parsedEx = parser(unicode(sentence))
        except:
            pdb.set_trace()
        # shown as: original token, dependency tag, head word, left dependents, right dependents
        for token in parsedEx:
            if token.dep_ in dp_pairs and not(token.orth_.lower() in stoplist) and not(token.head.orth_.lower() in stoplist):
                first = porter.stem(lmtzr.lemmatize(token.orth_.lower()))
                second = porter.stem(lmtzr.lemmatize(token.head.orth_.lower()))
                keyword = "_".join(sorted([first, second]))
                if keyword in freq_dict.keys():
                    freq_dict[keyword] = freq_dict[keyword] + 1
                else:
                    freq_dict[keyword] = 1
    
    
# texts = [[wordnet_lemmatizer.lemmatize(word) for word in document.lower().split() if word not in stoplist] for document in documents]


In [5]:

for example in documents:
    sentences = nltk.sent_tokenize(example)
    test = []
    for sentence in sentences:
        try:
            parsedEx = parser(unicode(sentence))
        except:
            pdb.set_trace()
        # shown as: original token, dependency tag, head word, left dependents, right dependents
        for token in parsedEx:
            if token.dep_ in dp_pairs and not(token.orth_.lower() in stoplist) and not(token.head.orth_.lower() in stoplist):
                first = porter.stem(lmtzr.lemmatize(token.orth_.lower()))
                second = porter.stem(lmtzr.lemmatize(token.head.orth_.lower()))
                keyword = "_".join(sorted([first, second]))
                if freq_dict[keyword] > 3:
                    test.append(keyword)
    texts.append(test)
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
# wordnet_lemmatizer = WordNetLemmatizer()
# texts = []
# for document in documents:
#     words = document.lower().replace("."," ").split()
#     text_sample = []
#     for word in words:
#         try:
#             if wordnet_lemmatizer.lemmatize(word) not in stoplist:
#                 text_sample.append(wordnet_lemmatizer.lemmatize(word))
#         except:
#             pdb.set_trace()
#     texts.append(text_sample)
# # texts = [[wordnet_lemmatizer.lemmatize(word) for word in document.lower().split() if word not in stoplist] for document in documents]
# dictionary = corpora.Dictionary(texts)
# corpus = [dictionary.doc2bow(text) for text in texts]

In [7]:
num_topics = 60
lda = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, update_every=1, chunksize=10000, passes=10)

In [8]:
for topic_num in xrange(num_topics):
    print "Topic " + str(topic_num) + ": " + str(lda.print_topics(num_topics)[topic_num])

Topic 0: (0, u'0.371*"drive_game" + 0.338*"camp_tent" + 0.054*"northern_serengeti" + 0.025*"entir_trip" + 0.021*"day_drive" + 0.020*"drive_night" + 0.017*"provid_servic" + 0.015*"lot_see" + 0.008*"\'s_easi" + 0.008*"fantast_time"')
Topic 1: (1, u'0.167*"great_view" + 0.067*"cross_river" + 0.036*"camp_locat" + 0.033*"enjoy_time" + 0.032*"anim_nois" + 0.027*"drink_water" + 0.024*"buffet_dinner" + 0.023*"previou_review" + 0.020*"guid_shatri" + 0.019*"big_deal"')
Topic 2: (2, u'0.077*"camp_great" + 0.053*"guid_safari" + 0.050*"amaz_time" + 0.048*"love_minut" + 0.044*"namiri_plain" + 0.033*"anim_spot" + 0.025*"beauti_room" + 0.022*"lodg_tree" + 0.022*"lake_lodg" + 0.020*"dunia_recommend"')
Topic 3: (3, u'0.192*"experi_wonder" + 0.111*"place_recommend" + 0.046*"safari_walk" + 0.040*"camp_sabora" + 0.039*"beauti_set" + 0.027*"asilia_properti" + 0.026*"best_way" + 0.025*"insect_repel" + 0.025*"bed_larg" + 0.024*"bed_doubl"')
Topic 4: (4, u'0.117*"last_night" + 0.069*"day_last" + 0.067*"lunch_p

In [9]:
probs = lda[corpus]

In [10]:
with open('topic_modelling_prob.txt','w') as f:
    for i in xrange(len(corpus)):
        prob_dict = dict(probs[i])
        for j in xrange(num_topics):
            if j in prob_dict.keys():
                print >> f,prob_dict[j],'\t',
            else:
                print >> f, '0','\t',
        print >> f,'\t'
f.close()

In [ ]:
texts_ = ["Im so happy when i have all GE production in my huose."]
test_ = []
try:
    parsedEx = parser(unicode(texts_[0]))
except:
    pdb.set_trace()
        # shown as: original token, dependency tag, head word, left dependents, right dependents
for token in parsedEx:
    if token.dep_ in dp_pairs and not(token.orth_.lower() in stoplist) and not(token.head.orth_.lower() in stoplist):
        first = porter.stem(lmtzr.lemmatize(token.orth_.lower()))
        second = porter.stem(lmtzr.lemmatize(token.head.orth_.lower()))
        keyword = "_".join(sorted([first, second]))
        if freq_dict[keyword] > 1:
            test_.append(keyword)
test_
corpus = [dictionary.doc2bow([sample]) for sample in test_]

In [36]:
# for topic_num in xrange(num_topics):
#     print "Topic " + str(topic_num) + ": " + str(lda.print_topics(num_topics)[topic_num])
import re
for topic_num in xrange(num_topics):
    topic_probs = re.split('[*+]',lda.show_topics(num_topics)[topic_num][1])
    sum_prob = float(topic_probs[0]) + float(topic_probs[2])
    if sum_prob > 0.3:
        print lda.show_topics(num_topics)[topic_num]
# lda.show_topics(num_topics)[0][1].split("*","+")

(0, u'0.371*"drive_game" + 0.338*"camp_tent" + 0.054*"northern_serengeti" + 0.025*"entir_trip" + 0.021*"day_drive" + 0.020*"drive_night" + 0.017*"provid_servic" + 0.015*"lot_see" + 0.008*"\'s_easi" + 0.008*"fantast_time"')
(3, u'0.192*"experi_wonder" + 0.111*"place_recommend" + 0.046*"safari_walk" + 0.040*"camp_sabora" + 0.039*"beauti_set" + 0.027*"asilia_properti" + 0.026*"best_way" + 0.025*"insect_repel" + 0.025*"bed_larg" + 0.024*"bed_doubl"')
(6, u'0.347*"hot_water" + 0.117*"bottl_water" + 0.042*";the_food" + 0.041*"lodg_main" + 0.033*"5_big" + 0.026*"bucket_list" + 0.019*"run_water" + 0.017*"big_thank" + 0.017*"trip_week" + 0.015*"chocol_hot"')
(12, u'0.188*"nation_park" + 0.177*"park_serengeti" + 0.135*"friendli_staff" + 0.047*"anniversari_celebr" + 0.046*"hotel_star" + 0.041*"\'m_sure" + 0.035*"anniversari_wed" + 0.031*"comfort_tent" + 0.023*"go_make" + 0.016*"experi_wilder"')
(13, u'0.267*"camp_sayari" + 0.086*"camp_fire" + 0.059*"day_first" + 0.042*"africa_east" + 0.034*"camp_